In [1]:
    if __name__ == '__main__':
        %reload_ext pidgin

In [2]:
    import ruamel.yaml as yaml, ast, builtins, IPython, typing as t, io

In [3]:
    __doc__ =\
`import pidgin`; `pidgin.specification.yaml` allows __Python__ data structures to be represented in `yaml` syntax.  Also, we establish the
ability for literate cells supply yaml front matter; front matter exports as metadata in the display & can be used by templated
strings.
    
>>> assert """
... a: b""" == "\na: b"

Adding horizontal rule in __Markdown__ or a `yaml` stream demarcation consumes the string as yaml.

The example below creates a dictionary from a string.
>>> assert """---
... a: b""" == {'a': 'b'}


The next example creates a list from a string.
>>> assert """---
... [a, b]""" == ['a', 'b']

For more information on yaml refer to...

In [4]:
    class Yaml(ast.NodeTransformer):
        def visit_Str(Emojize, node):
            if node.s.lstrip().startswith('\\') or (node.s.lstrip().startswith('---') and '\n' in node.s):
                e = ast.parse("""next(__import__('ruamel.yaml').yaml.YAML().load_all(__import__('io').StringIO()))""").body[0].value
                node.s = node.s.lstrip('\\')
                e.args[0].args[0].args = [node]
                return e
            return node

In [5]:
`yaml` front matter is a convention in modern blogging, and RMarkdown.

In [6]:
    class FrontMatter:
        def front_matter(Weave, str) -> t.Tuple[str, t.Dict]: 
            import ruamel.yaml
            if str.startswith('---'):
                if '\n---' in str:
                    data, sep, body = str.lstrip('---').partition('---\n')
                    data = ruamel.yaml.YAML().load(__import__('io').StringIO(
                        Weave.environment.from_string(data).render({**vars(builtins), **Weave.user_ns})))
                    if isinstance(data, dict): return body, data
            return str, {}

In [7]:
    def load_ipython_extension(shell: IPython.InteractiveShell):
        shell.ast_transformers.insert(0, Yaml())

In [8]:
    def unload_ipython_extension(shell: IPython.InteractiveShell):
        for i, object in enumerate(shell.ast_transformers):
            if isinstance(object, Yaml): shell.ast_transformers.pop(i); break